# Define graph-level inputs and outputs

## Introduction

When constructing complex workflows, you may encounter tasks that share input parameters (e.g. `code`). Also, you may wish to reorder or rename internal task outputs at the top level (e.g. `wg.outputs.optimized_stuff` instead of `wg.outputs.optimize.stuff`).

`WorkGraph` allows you to define **graph-level inputs and outputs** to:

- **Share inputs** across multiple tasks in a graph.
- **Aggregate outputs** from internal tasks, optionally organizing or renaming them.
- **Simplify the interface** by exposing shared parameters to users (retaining the flexibility of internal task parameter definitions).

In [8]:
from aiida_workgraph import WorkGraph, task
from aiida import load_profile

_ = load_profile()

## Defining graph-level inputs and outputs

In [9]:
with WorkGraph("GraphLevelInput") as wg:
    # Define graph-level inputs
    wg.inputs = dict.fromkeys(["x", "y", "z"])

    # Define the tasks
    the_sum = wg.inputs.x + wg.inputs.y
    the_product = the_sum * wg.inputs.z

    # Define graph-level outputs
    wg.outputs.sum = the_sum
    wg.outputs.product = the_product


wg.to_html()

### Run the workgraph


In [10]:
wg.submit(
    {
        "graph_inputs": {
            "x": 1,
            "y": 2,
            "z": 3,
        },
    },
    wait=True,
)

print("\nGraph-level outputs:")
print("  Sum:", wg.outputs.sum.value)
print("  Product:", wg.outputs.product.value)

WorkGraph process created, PK: 11826
Process 11826 finished with state: FINISHED

Graph-level outputs:
  Sum: uuid: 54f4a3a6-a7e7-4fd6-89fe-2b2049cd46b9 (pk: 11830) value: 3
  Product: uuid: 729bf499-4c6f-4d0e-b3f9-780da15cf775 (pk: 11834) value: 9


## Providing graph-level inputs metadata

Graph-level inputs can also be defined using `wg.add_input(...)`. The method allows you to provide an identifier (e.g. `workgraph.int`) to the input, which is used for validation. Also, if you use the AiiDA GUI, providing an `identifier` will associate the input with a GUI component, allowing users to interact with the input in a more user-friendly type-specific way.

In [11]:
with WorkGraph("GraphLevelInputMetadata") as wg:
    wg.add_input("workgraph.int", "x")  # validated as an integer
    wg.add_input("workgraph.int", "y")
    wg.add_input("workgraph.int", "z")
    wg.outputs.result = (wg.inputs.x + wg.inputs.y) * wg.inputs.z

In the future, further details may be added when defining inputs, e.g., default values, descriptions, help messages, etc.

## Nested graph-level inputs and outputs

Graph-level inputs and outputs can be **nested** allowing you to group related parameters and results. Here we're using the `add_task` interface to more clearly define the names of our tasks.

In [12]:
@task
def add(x, y):
    return x + y


@task
def multiply(x, y):
    return x * y


with WorkGraph("GraphLevelInputOutputNested") as wg:
    wg.inputs = {
        "add": {
            "first": dict.fromkeys(["x", "y"]),
            "second": dict.fromkeys(["x", "y"]),
        },
        "multiply": dict.fromkeys(["factor"]),
    }

    first_sum = wg.inputs.add.first.x + wg.inputs.add.first.y
    second_sum = wg.inputs.add.second.x + wg.inputs.add.second.y
    third_sum = first_sum + second_sum
    product = third_sum * wg.inputs.multiply.factor

    wg.outputs.results = {
        "sums": {
            "first": first_sum,
            "second": second_sum,
            "third": third_sum,
        },
        "product": product,
    }


wg.to_html()

### Run the workgraph

We can now run our workgraph with a clear input layout. Note that `WorkGraph` will automatically serialize the raw Python data into the corresponding AiiDA Data nodes (e.g., an `int` becomes `orm.Int`, a `str` becomes `orm.Str`, etc.) before execution. The exact serialization logic and all supported types (and how to register your own custom serializers) are described in detail in the **Data Serialization** section.

In [13]:
wg.submit(
    inputs={
        "graph_inputs": {
            "add": {
                "first": {
                    "x": 1,
                    "y": 2,
                },
                "second": {
                    "x": 3,
                    "y": 4,
                },
            },
            "multiply": {
                "factor": 5,
            },
        }
    },
    wait=True,
)

print("\nResults:")
print("  Sums:")
print("    First:", wg.outputs.results.sums.first.value)
print("    Second:", wg.outputs.results.sums.second.value)
print("    Third:", wg.outputs.results.sums.third.value)
print("  Product:", wg.outputs.results.product.value)

WorkGraph process created, PK: 11840
Process 11840 finished with state: FINISHED

Results:
  Sums:
    First: uuid: 68fe9b1a-d869-499a-82b9-70c5556cc258 (pk: 11844) value: 3
    Second: uuid: 65f79486-a881-488b-ae7a-92caaeb33b87 (pk: 11848) value: 7
    Third: uuid: 50027e21-ff83-412c-bab4-86b089b71406 (pk: 11852) value: 10
  Product: uuid: be032d48-c884-4325-b219-527ce49ca365 (pk: 11856) value: 50


When we inspect the outputs of the `WorkGraph` process, we see `sums` and `product` are grouped under the `results` output.

In [14]:
!verdi process show {wg.pk}

Property     Value
-----------  --------------------------------------
type         WorkGraph<GraphLevelInputOutputNested>
state        Finished [0]
pk           11840
uuid         3c10d78f-7c8d-4a12-a8ee-5d2b4f51866c
label        GraphLevelInputOutputNested
description
ctime        2025-07-14 15:08:32.992863+00:00
mtime        2025-07-14 15:08:35.716053+00:00

Inputs                                         PK     Type
---------------------------------------------  -----  ------
workgraph_data
    meta_sockets
        graph_inputs
            sockets
                add
                    sockets
                        first
                            sockets
                                x
                                    property
                                        value  11835  Int
                                y
                                    property
                                        value  11836  Int
                        second
                        

## Summary

In this section, you learned how to:

- Use `wg.inputs = {...}` to define many inputs at once, or to define nested (namespaced) inputs to group related parameters
- Use `wg.add_input(...)` to define a graph-level input and provide additional metadata (e.g., type validation)
- Use graph-level inputs in tasks (`wg.inputs.<name>`)
- Use `wg.outputs.<name>` to expose graph-level outputs from internal tasks